Get all files from histo into cassandra cluster

In [2]:

import pandas as pd
from pathlib import Path

# parse all files from histo_velib\paris csv files and aggregate in a dataframe
paris_part1 = Path("../histo_velib\challenge\part_1\paris\stations")
paris_part2 = Path("../histo_velib\challenge\part_2\paris\stations")

def df_generator(dir):
    if issubclass(type(dir),Path) and dir.is_dir():
        for file in dir.iterdir():
            # extract the velib station id from filename
            station_id = file.name.split("-")[0]
            # create a dataframe from csv
            df = pd.read_csv(file)

            # create a series for the station id entry in the dataframe
            df["station_id"] = pd.Series(data=[station_id for row in range(0,len(df.index))], index=df.index)
            yield df

def get_first(generator):
    for i,df in enumerate(generator):
        if (i == 0):
            return df
        
def remove_unchanged(row):
    return row

df = get_first(df_generator(paris_part1))

def convert_to_date(col,name):
    if col.name == name:
        return pd.to_datetime(col)
    else:
        return col

df = df.apply(convert_to_date,args=["moment"]).set_index("moment").sort_index().reset_index()

#type(df.index)
print(len(df))

def filterout(row,df):
    #print(row.name) 
    return row

#for index in range(0,len(df)):
    
    
df = df.head().apply(filterout,axis=1,args=[df.head()])
df

12917


,moment,bikes,spaces,station_id
0,2016-08-01 10:51:44,7,13,00901
1,2016-08-01 11:01:58,7,13,00901
2,2016-08-01 11:12:02,7,13,00901
3,2016-08-01 11:14:46,6,14,00901
4,2016-08-01 11:24:50,6,14,00901


In [ ]:
# just for the exercise : concat all dataset in a full indexed dataframe
"""
full_df = pd.concat([df.filter(remove_unchanged)\
                       .apply(convert_to_date,args=["moment"])\
                       .set_index(moment) \
                     for df in df_generator(paris_part1)] )\
            .concat([df.filter(remove_unchanged)\
                       .apply(convert_to_date,args=["moment"])\
                       .set_index(moment) \
                     for df in df_generator(paris_part2)] )
full_df.head()
"""

# insert data in cassandra db

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(["84.39.48.220","84.39.45.143","84.39.45.149","84.39.48.102"])
session = cluster.connect("velib_db_test")


In [6]:
insert = "INSERT INTO station (last_update, available_bikes,available_bike_stands,number) VALUES (?,?,?,?)"
stmt = session.prepare(insert)

def insert_into_db(row):
    session.execute(insert,list(row))

df.apply(lambda row: session.execute(insert,list(row)), axis=1)
#for station in rows:
#    print(" , ".join([str(station.number), str(station.position)]))

TypeError: ('not all arguments converted during string formatting', 'occurred at index 0')